In [14]:
# USE THIS CELL, IT HAS THE FIXED LATEX AUTOMATED TABLE CODE 
# RXJ0911
import numpy as np
import pandas as pd
from astropy.io import fits
import os

# Function to calculate distance
def process_file_group(output_file, fits_file, input_file, n):
    observed_positions, observed_flux_ratios = read_observed_positions(input_file)
    predicted_positions, predicted_magnifications = read_predicted_positions(output_file)
    
    paired_positions = pair_positions(observed_positions, predicted_positions, predicted_magnifications)
    
    # Calculate Delta RMS of Image Positions correctly
    squared_distances = 0.0
    for observed, predicted, _ in paired_positions:
        distance = calculate_distance(*observed, *predicted)
        squared_distances += distance ** 2  # Summing squared distances
    
    mean_squared_distances = squared_distances / len(paired_positions)
    delta_rms = np.sqrt(mean_squared_distances)  # Taking square root of the mean of squared distances
    delta_rms *= 1000  # Convert to mas
    
    # Order predicted positions and magnifications based on observed order
    predicted_positions_ordered = [pair[1] for pair in paired_positions]
    predicted_magnifications_ordered = [pair[2] for pair in paired_positions]
    
    # Determine the index of the "Green Image"
    green_index = 0  # CHOOSE WHICH IMAGE TO NORMALISE BY HERE !!!!!!!!!!!!!!!!!!!!!!!

    # Predicted Flux Ratios
    predicted_flux_ratios = [m / predicted_magnifications_ordered[green_index] for m in predicted_magnifications_ordered]

    with fits.open(fits_file) as hdul:
        mag = hdul[0].data[6]
    
    pixel_coords = [(int((x + mag.shape[1] // 2 * n) / n), int((y + mag.shape[0] // 2 * n) / n)) for x, y in observed_positions]
    values = [mag[y, x] for x, y in pixel_coords]
    
    abs_values = [1 / abs(value) for value in values]
    predicted_flux_ratios_observed = [abs_values[i] / abs_values[green_index] for i in range(len(abs_values))]

    # Calculate RMS excluding the green image
    squared_diffs_pred = [(pred - obs) ** 2 for i, (pred, obs) in enumerate(zip(predicted_flux_ratios, observed_flux_ratios)) if i != green_index]
    predicted_flux_ratios_rms = np.sqrt(sum(squared_diffs_pred) / len(squared_diffs_pred))

    squared_diffs_obs = [(pred - obs) ** 2 for i, (pred, obs) in enumerate(zip(predicted_flux_ratios_observed, observed_flux_ratios)) if i != green_index]
    predicted_flux_ratios_observed_rms = np.sqrt(sum(squared_diffs_obs) / len(squared_diffs_obs))

    return {
        'delta_rms': delta_rms,
        'predicted_flux_ratios_rms': predicted_flux_ratios_rms,
        'predicted_flux_ratios_observed_rms': predicted_flux_ratios_observed_rms
    }

# Function to extract chi^2 and lens models from optresult.dat file
def extract_chi2_and_lens_profiles(optresult_path):
    chi2_value = None
    profiles_used = []
    sie_count = 0

    with open(optresult_path, 'r') as file:
        lines = file.readlines()

    found_chi2 = False
    for line in lines:
        if line.startswith("chi^2"):
            chi2_value = float(line.split()[2])
            found_chi2 = True
            profiles_used = []
            sie_count = 0
        elif found_chi2:
            if line.startswith("lens   sie"):
                profiles_used.append("SIE")
                sie_count += 1
            elif line.startswith("lens   pow"):
                profiles_used.append("POW")
            elif line.startswith("lens   anfw"):
                profiles_used.append("NFW")
            elif line.startswith("lens   pert"):
                profiles_used.append("Shear")

    if chi2_value is None:
        raise ValueError("No chi^2 value found in the file.")

    profiles_used = list(set(profiles_used))
    return chi2_value, profiles_used, sie_count

# Function to calculate free parameters based on lens profiles
def calculate_free_parameters(profiles_used, sie_count):
    profile_params = {
        "SIE": 5,
        "POW": 6,
        "NFW": 6,
        "Shear": 5
    }
    total_params = sum(profile_params[profile] for profile in profiles_used)
    if sie_count == 2:
        total_params += 5
    return total_params

# Function to calculate BIC (Bayesian Information Criterion)
def calculate_bic(chi2, n_data_points, n_free_params):
    return chi2 + n_free_params * np.log(n_data_points)

# Function to compute BIC from optresult file
def compute_bic(optresult_path):
    chi2_value, profiles_used, sie_count = extract_chi2_and_lens_profiles(optresult_path)
    n_free_params = calculate_free_parameters(profiles_used, sie_count)
    
    # Adjust n_data_points based on "PF" in the filename
    if "PF" in optresult_path:
        n_data_points = 4 * 2 + 3
    else:
        n_data_points = 4 * 2
    
    bic_value = calculate_bic(chi2_value, n_data_points, n_free_params)
    return bic_value

# Main function to process multiple files
def process_files(input_file, output_files, fits_files, optresult_files, n):
    results = []
    latex_rows = []
    bics = []  # Store BIC values separately

    for output_file, fits_file, optresult_file in zip(output_files, fits_files, optresult_files):
        # Process output and fits files
        result = process_file_group(output_file, fits_file, input_file, n)
        results.append(result)
        
        # Process optresult files separately for BIC calculation
        bic = compute_bic(optresult_file)
        bics.append(bic)
        
        # Create LaTeX row
        model_name, constraint_name = determine_model_and_constraint(output_file)
        latex_row = f"{model_name} & {constraint_name} & {result['delta_rms']:.1f} & {result['predicted_flux_ratios_observed_rms']:.2f} & {result['predicted_flux_ratios_rms']:.2f} & {bic:.1f} \\\\"
        latex_rows.append(latex_row)
    
    # Combine results into a DataFrame
    df_results = pd.DataFrame(results)
    df_results['bic'] = bics  # Add BIC values to the DataFrame
    
    return df_results, latex_rows

# Determine model and constraint names from the filename
def determine_model_and_constraint(file_name):
    model_map = {'S': 'SIE', 'P': 'POW', 'N': 'NFW'}
    
    base_name = os.path.basename(file_name)
    model_key = base_name[3]
    constraint_key = base_name[4:6] if base_name[4:6] in ['PF'] else base_name[4]
    
    model_name = model_map.get(model_key, '')
    if 'R' in base_name:
        model_name += ' + Shear'
    if 'G' in base_name:
        model_name += ' + G2'
    
    constraint_name = 'QSO Pos + FR' if constraint_key == 'PF' else 'QSO Pos'
    
    return model_name, constraint_name

# Example usage with filenames
Input_file = "RXJ0911/SPFC/obs_point.dat"

Output_files = [
    "RXJ0911/SPC/outSP_point.dat",
    "RXJ0911/SPC/outSPR_point.dat",
    "RXJ0911/SPC/outSPG_point.dat",
    "RXJ0911/SPC/outSPGR_point.dat",
    "RXJ0911/SPFC/outSPF_point.dat",
    "RXJ0911/SPFC/outSPFR_point.dat",
    "RXJ0911/SPFC/outSPFG_point.dat",
    "RXJ0911/SPFC/outSPFGR_point.dat",
    "RXJ0911/PPC/outPP_point.dat",
    "RXJ0911/PPC/outPPR_point.dat",
    "RXJ0911/PPC/outPPG_point.dat",
    "RXJ0911/PPC/outPPGR_point.dat",
    "RXJ0911/PPFC/outPPF_point.dat",
    "RXJ0911/PPFC/outPPFR_point.dat",
    "RXJ0911/PPFC/outPPFG_point.dat",
    "RXJ0911/PPFC/outPPFGR_point.dat",
    "RXJ0911/NPC/outNP_point.dat",
    "RXJ0911/NPC/outNPR_point.dat",
    "RXJ0911/NPC/outNPG_point.dat",
    "RXJ0911/NPC/outNPGR_point.dat",
    "RXJ0911/NPFC/outNPF_point.dat",
    "RXJ0911/NPFC/outNPFR_point.dat",
    "RXJ0911/NPFC/outNPFG_point.dat",
    "RXJ0911/NPFC/outNPFGR_point.dat",
]

Fits_files = [
    "RXJ0911/SPC/outSP_lens.fits",
    "RXJ0911/SPC/outSPR_lens.fits",
    "RXJ0911/SPC/outSPG_lens.fits",
    "RXJ0911/SPC/outSPGR_lens.fits",
    "RXJ0911/SPFC/outSPF_lens.fits",
    "RXJ0911/SPFC/outSPFR_lens.fits",
    "RXJ0911/SPFC/outSPFG_lens.fits",
    "RXJ0911/SPFC/outSPFGR_lens.fits",
    "RXJ0911/PPC/outPP_lens.fits",
    "RXJ0911/PPC/outPPR_lens.fits",
    "RXJ0911/PPC/outPPG_lens.fits",
    "RXJ0911/PPC/outPPGR_lens.fits",
    "RXJ0911/PPFC/outPPF_lens.fits",
    "RXJ0911/PPFC/outPPFR_lens.fits",
    "RXJ0911/PPFC/outPPFG_lens.fits",
    "RXJ0911/PPFC/outPPFGR_lens.fits",
    "RXJ0911/NPC/outNP_lens.fits",
    "RXJ0911/NPC/outNPR_lens.fits",
    "RXJ0911/NPC/outNPG_lens.fits",
    "RXJ0911/NPC/outNPGR_lens.fits",
    "RXJ0911/NPFC/outNPF_lens.fits",
    "RXJ0911/NPFC/outNPFR_lens.fits",
    "RXJ0911/NPFC/outNPFG_lens.fits",
    "RXJ0911/NPFC/outNPFGR_lens.fits"
]

Optresult_files = [
    "RXJ0911/SPC/outSP_optresult.dat",
    "RXJ0911/SPC/outSPR_optresult.dat",
    "RXJ0911/SPC/outSPG_optresult.dat",
    "RXJ0911/SPC/outSPGR_optresult.dat",
    "RXJ0911/SPFC/outSPF_optresult.dat",
    "RXJ0911/SPFC/outSPFR_optresult.dat",
    "RXJ0911/SPFC/outSPFG_optresult.dat",
    "RXJ0911/SPFC/outSPFGR_optresult.dat",
    "RXJ0911/PPC/outPP_optresult.dat",
    "RXJ0911/PPC/outPPR_optresult.dat",
    "RXJ0911/PPC/outPPG_optresult.dat",
    "RXJ0911/PPC/outPPGR_optresult.dat",
    "RXJ0911/PPFC/outPPF_optresult.dat",
    "RXJ0911/PPFC/outPPFR_optresult.dat",
    "RXJ0911/PPFC/outPPFG_optresult.dat",
    "RXJ0911/PPFC/outPPFGR_optresult.dat",
    "RXJ0911/NPC/outNP_optresult.dat",
    "RXJ0911/NPC/outNPR_optresult.dat",
    "RXJ0911/NPC/outNPG_optresult.dat",
    "RXJ0911/NPC/outNPGR_optresult.dat",
    "RXJ0911/NPFC/outNPF_optresult.dat",
    "RXJ0911/NPFC/outNPFR_optresult.dat",
    "RXJ0911/NPFC/outNPFG_optresult.dat",
    "RXJ0911/NPFC/outNPFGR_optresult.dat"
]

n = 0.0025  # Example scale factor
results_df, latex_rows = process_files(Input_file, Output_files, Fits_files, Optresult_files, n)

# Print formatted results
for i, output_file in enumerate(Output_files):
    print(f"Model: {output_file}")
    print(f"Delta RMS of Image Positions: {results_df.iloc[i]['delta_rms']:.3f}")
    print(f"Delta RMS of flux ratios at the observed position: {results_df.iloc[i]['predicted_flux_ratios_observed_rms']:.3f}")
    print(f"Delta RMS of flux ratios at the predicted position: {results_df.iloc[i]['predicted_flux_ratios_rms']:.3f}")
    print(f"BIC: {results_df.iloc[i]['bic']:.3f}\n")

# Print LaTeX-formatted rows
print("LaTeX Table Rows:")
for row in latex_rows:
    print(row)

Model: RXJ0911/SPC/outSP_point.dat
Delta RMS of Image Positions: 66.832
Delta RMS of flux ratios at the observed position: 0.265
Delta RMS of flux ratios at the predicted position: 0.503
BIC: 819.432

Model: RXJ0911/SPC/outSPR_point.dat
Delta RMS of Image Positions: 2.299
Delta RMS of flux ratios at the observed position: 0.602
Delta RMS of flux ratios at the predicted position: 0.612
BIC: 31.592

Model: RXJ0911/SPC/outSPG_point.dat
Delta RMS of Image Positions: 49.763
Delta RMS of flux ratios at the observed position: 0.093
Delta RMS of flux ratios at the predicted position: 0.514
BIC: 468.048

Model: RXJ0911/SPC/outSPGR_point.dat
Delta RMS of Image Positions: 0.000
Delta RMS of flux ratios at the observed position: 0.548
Delta RMS of flux ratios at the predicted position: 0.557
BIC: 31.192

Model: RXJ0911/SPFC/outSPF_point.dat
Delta RMS of Image Positions: 66.802
Delta RMS of flux ratios at the observed position: 0.268
Delta RMS of flux ratios at the predicted position: 0.509
BIC: 84

In [15]:
# USE THIS CELL, IT HAS THE FIXED LATEX AUTOMATED TABLE CODE 
# PSJ1606
import numpy as np
import pandas as pd
from astropy.io import fits
import os

# Function to calculate distance
def process_file_group(output_file, fits_file, input_file, n):
    observed_positions, observed_flux_ratios = read_observed_positions(input_file)
    predicted_positions, predicted_magnifications = read_predicted_positions(output_file)
    
    paired_positions = pair_positions(observed_positions, predicted_positions, predicted_magnifications)
    
    # Calculate Delta RMS of Image Positions correctly
    squared_distances = 0.0
    for observed, predicted, _ in paired_positions:
        distance = calculate_distance(*observed, *predicted)
        squared_distances += distance ** 2  # Summing squared distances
    
    mean_squared_distances = squared_distances / len(paired_positions)
    delta_rms = np.sqrt(mean_squared_distances)  # Taking square root of the mean of squared distances
    delta_rms *= 1000  # Convert to mas
    
    # Order predicted positions and magnifications based on observed order
    predicted_positions_ordered = [pair[1] for pair in paired_positions]
    predicted_magnifications_ordered = [pair[2] for pair in paired_positions]
    
    # Determine the index of the "Green Image"
    green_index = 0  # CHOOSE WHICH IMAGE TO NORMALISE BY HERE !!!!!!!!!!!!!!!!!!!!!!!

    # Predicted Flux Ratios
    predicted_flux_ratios = [m / predicted_magnifications_ordered[green_index] for m in predicted_magnifications_ordered]

    with fits.open(fits_file) as hdul:
        mag = hdul[0].data[6]
    
    pixel_coords = [(int((x + mag.shape[1] // 2 * n) / n), int((y + mag.shape[0] // 2 * n) / n)) for x, y in observed_positions]
    values = [mag[y, x] for x, y in pixel_coords]
    
    abs_values = [1 / abs(value) for value in values]
    predicted_flux_ratios_observed = [abs_values[i] / abs_values[green_index] for i in range(len(abs_values))]

    # Calculate RMS excluding the green image
    squared_diffs_pred = [(pred - obs) ** 2 for i, (pred, obs) in enumerate(zip(predicted_flux_ratios, observed_flux_ratios)) if i != green_index]
    predicted_flux_ratios_rms = np.sqrt(sum(squared_diffs_pred) / len(squared_diffs_pred))

    squared_diffs_obs = [(pred - obs) ** 2 for i, (pred, obs) in enumerate(zip(predicted_flux_ratios_observed, observed_flux_ratios)) if i != green_index]
    predicted_flux_ratios_observed_rms = np.sqrt(sum(squared_diffs_obs) / len(squared_diffs_obs))

    return {
        'delta_rms': delta_rms,
        'predicted_flux_ratios_rms': predicted_flux_ratios_rms,
        'predicted_flux_ratios_observed_rms': predicted_flux_ratios_observed_rms
    }

# Function to extract chi^2 and lens models from optresult.dat file
def extract_chi2_and_lens_profiles(optresult_path):
    chi2_value = None
    profiles_used = []
    sie_count = 0

    with open(optresult_path, 'r') as file:
        lines = file.readlines()

    found_chi2 = False
    for line in lines:
        if line.startswith("chi^2"):
            chi2_value = float(line.split()[2])
            found_chi2 = True
            profiles_used = []
            sie_count = 0
        elif found_chi2:
            if line.startswith("lens   sie"):
                profiles_used.append("SIE")
                sie_count += 1
            elif line.startswith("lens   pow"):
                profiles_used.append("POW")
            elif line.startswith("lens   anfw"):
                profiles_used.append("NFW")
            elif line.startswith("lens   pert"):
                profiles_used.append("Shear")

    if chi2_value is None:
        raise ValueError("No chi^2 value found in the file.")

    profiles_used = list(set(profiles_used))
    return chi2_value, profiles_used, sie_count

# Function to calculate free parameters based on lens profiles
def calculate_free_parameters(profiles_used, sie_count):
    profile_params = {
        "SIE": 5,
        "POW": 6,
        "NFW": 6,
        "Shear": 5
    }
    total_params = sum(profile_params[profile] for profile in profiles_used)
    if sie_count == 2:
        total_params += 5
    return total_params

# Function to calculate BIC (Bayesian Information Criterion)
def calculate_bic(chi2, n_data_points, n_free_params):
    return chi2 + n_free_params * np.log(n_data_points)

# Function to compute BIC from optresult file
def compute_bic(optresult_path):
    chi2_value, profiles_used, sie_count = extract_chi2_and_lens_profiles(optresult_path)
    n_free_params = calculate_free_parameters(profiles_used, sie_count)
    
    # Adjust n_data_points based on "PF" in the filename
    if "PF" in optresult_path:
        n_data_points = 4 * 2 + 3
    else:
        n_data_points = 4 * 2
    
    bic_value = calculate_bic(chi2_value, n_data_points, n_free_params)
    return bic_value

# Main function to process multiple files
def process_files(input_file, output_files, fits_files, optresult_files, n):
    results = []
    latex_rows = []
    bics = []  # Store BIC values separately

    for output_file, fits_file, optresult_file in zip(output_files, fits_files, optresult_files):
        # Process output and fits files
        result = process_file_group(output_file, fits_file, input_file, n)
        results.append(result)
        
        # Process optresult files separately for BIC calculation
        bic = compute_bic(optresult_file)
        bics.append(bic)
        
        # Create LaTeX row
        model_name, constraint_name = determine_model_and_constraint(output_file)
        latex_row = f"{model_name} & {constraint_name} & {result['delta_rms']:.1f} & {result['predicted_flux_ratios_observed_rms']:.2f} & {result['predicted_flux_ratios_rms']:.2f} & {bic:.1f} \\\\"
        latex_rows.append(latex_row)
    
    # Combine results into a DataFrame
    df_results = pd.DataFrame(results)
    df_results['bic'] = bics  # Add BIC values to the DataFrame
    
    return df_results, latex_rows

# Determine model and constraint names from the filename
def determine_model_and_constraint(file_name):
    model_map = {'S': 'SIE', 'P': 'POW', 'N': 'NFW'}
    
    base_name = os.path.basename(file_name)
    model_key = base_name[3]
    constraint_key = base_name[4:6] if base_name[4:6] in ['PF'] else base_name[4]
    
    model_name = model_map.get(model_key, '')
    if 'R' in base_name:
        model_name += ' + Shear'
    if 'G' in base_name:
        model_name += ' + G2'
    
    constraint_name = 'QSO Pos + FR' if constraint_key == 'PF' else 'QSO Pos'
    
    return model_name, constraint_name

# Example usage with filenames
Input_file = "PSJ1606/SPFC/Lobs_point.dat"

Output_files = [
    "PSJ1606/SPC/outSP_point.dat",
    "PSJ1606/SPC/outSPR_point.dat",
    "PSJ1606/SPC/outSPG_point.dat",
    "PSJ1606/SPC/outSPGR_point.dat",
    "PSJ1606/SPFC/outSPF_point.dat",
    "PSJ1606/SPFC/outSPFR_point.dat",
    "PSJ1606/SPFC/outSPFG_point.dat",
    "PSJ1606/SPFC/outSPFGR_point.dat",
    "PSJ1606/PPC/outPP_point.dat",
    "PSJ1606/PPC/outPPR_point.dat",
    "PSJ1606/PPC/outPPG_point.dat",
    "PSJ1606/PPC/outPPGR_point.dat",
    "PSJ1606/PPFC/outPPF_point.dat",
    "PSJ1606/PPFC/outPPFR_point.dat",
    "PSJ1606/PPFC/outPPFG_point.dat",
    "PSJ1606/PPFC/outPPFGR_point.dat",
    "PSJ1606/NPC/outNP_point.dat",
    "PSJ1606/NPC/outNPR_point.dat",
    "PSJ1606/NPC/outNPG_point.dat",
    "PSJ1606/NPC/outNPGR_point.dat",
    "PSJ1606/NPFC/outNPF_point.dat",
    "PSJ1606/NPFC/outNPFR_point.dat",
    "PSJ1606/NPFC/outNPFG_point.dat",
    "PSJ1606/NPFC/outNPFGR_point.dat"
]

Fits_files = [
    "PSJ1606/SPC/outSP_lens.fits",
    "PSJ1606/SPC/outSPR_lens.fits",
    "PSJ1606/SPC/outSPG_lens.fits",
    "PSJ1606/SPC/outSPGR_lens.fits",
    "PSJ1606/SPFC/outSPF_lens.fits",
    "PSJ1606/SPFC/outSPFR_lens.fits",
    "PSJ1606/SPFC/outSPFG_lens.fits",
    "PSJ1606/SPFC/outSPFGR_lens.fits",
    "PSJ1606/PPC/outPP_lens.fits",
    "PSJ1606/PPC/outPPR_lens.fits",
    "PSJ1606/PPC/outPPG_lens.fits",
    "PSJ1606/PPC/outPPGR_lens.fits",
    "PSJ1606/PPFC/outPPF_lens.fits",
    "PSJ1606/PPFC/outPPFR_lens.fits",
    "PSJ1606/PPFC/outPPFG_lens.fits",
    "PSJ1606/PPFC/outPPFGR_lens.fits",
    "PSJ1606/NPC/outNP_lens.fits",
    "PSJ1606/NPC/outNPR_lens.fits",
    "PSJ1606/NPC/outNPG_lens.fits",
    "PSJ1606/NPC/outNPGR_lens.fits",
    "PSJ1606/NPFC/outNPF_lens.fits",
    "PSJ1606/NPFC/outNPFR_lens.fits",
    "PSJ1606/NPFC/outNPFG_lens.fits",
    "PSJ1606/NPFC/outNPFGR_lens.fits"
]

Optresult_files = [
    "PSJ1606/SPC/outSP_optresult.dat",
    "PSJ1606/SPC/outSPR_optresult.dat",
    "PSJ1606/SPC/outSPG_optresult.dat",
    "PSJ1606/SPC/outSPGR_optresult.dat",
    "PSJ1606/SPFC/outSPF_optresult.dat",
    "PSJ1606/SPFC/outSPFR_optresult.dat",
    "PSJ1606/SPFC/outSPFG_optresult.dat",
    "PSJ1606/SPFC/outSPFGR_optresult.dat",
    "PSJ1606/PPC/outPP_optresult.dat",
    "PSJ1606/PPC/outPPR_optresult.dat",
    "PSJ1606/PPC/outPPG_optresult.dat",
    "PSJ1606/PPC/outPPGR_optresult.dat",
    "PSJ1606/PPFC/outPPF_optresult.dat",
    "PSJ1606/PPFC/outPPFR_optresult.dat",
    "PSJ1606/PPFC/outPPFG_optresult.dat",
    "PSJ1606/PPFC/outPPFGR_optresult.dat",
    "PSJ1606/NPC/outNP_optresult.dat",
    "PSJ1606/NPC/outNPR_optresult.dat",
    "PSJ1606/NPC/outNPG_optresult.dat",
    "PSJ1606/NPC/outNPGR_optresult.dat",
    "PSJ1606/NPFC/outNPF_optresult.dat",
    "PSJ1606/NPFC/outNPFR_optresult.dat",
    "PSJ1606/NPFC/outNPFG_optresult.dat",
    "PSJ1606/NPFC/outNPFGR_optresult.dat"
]

n = 0.0025  # Example scale factor
results_df, latex_rows = process_files(Input_file, Output_files, Fits_files, Optresult_files, n)

# Print formatted results
for i, output_file in enumerate(Output_files):
    print(f"Model: {output_file}")
    print(f"Delta RMS of Image Positions: {results_df.iloc[i]['delta_rms']:.3f}")
    print(f"Delta RMS of flux ratios at the observed position: {results_df.iloc[i]['predicted_flux_ratios_observed_rms']:.3f}")
    print(f"Delta RMS of flux ratios at the predicted position: {results_df.iloc[i]['predicted_flux_ratios_rms']:.3f}")
    print(f"BIC: {results_df.iloc[i]['bic']:.3f}\n")

# Print LaTeX-formatted rows
print("LaTeX Table Rows:")
for row in latex_rows:
    print(row)

Model: PSJ1606/SPC/outSP_point.dat
Delta RMS of Image Positions: 45.026
Delta RMS of flux ratios at the observed position: 0.253
Delta RMS of flux ratios at the predicted position: 0.252
BIC: 351.631

Model: PSJ1606/SPC/outSPR_point.dat
Delta RMS of Image Positions: 2.910
Delta RMS of flux ratios at the observed position: 0.164
Delta RMS of flux ratios at the predicted position: 0.165
BIC: 27.599

Model: PSJ1606/SPC/outSPG_point.dat
Delta RMS of Image Positions: 16.500
Delta RMS of flux ratios at the observed position: 0.325
Delta RMS of flux ratios at the predicted position: 0.317
BIC: 131.779

Model: PSJ1606/SPC/outSPGR_point.dat
Delta RMS of Image Positions: 0.000
Delta RMS of flux ratios at the observed position: 0.204
Delta RMS of flux ratios at the predicted position: 0.205
BIC: 31.192

Model: PSJ1606/SPFC/outSPF_point.dat
Delta RMS of Image Positions: 45.395
Delta RMS of flux ratios at the observed position: 0.251
Delta RMS of flux ratios at the predicted position: 0.248
BIC: 64

In [17]:
# USE THIS CELL, IT HAS THE FIXED LATEX AUTOMATED TABLE CODE 
# WFI2033
import numpy as np
import pandas as pd
from astropy.io import fits
import os

# Function to calculate distance
def process_file_group(output_file, fits_file, input_file, n):
    observed_positions, observed_flux_ratios = read_observed_positions(input_file)
    predicted_positions, predicted_magnifications = read_predicted_positions(output_file)
    
    paired_positions = pair_positions(observed_positions, predicted_positions, predicted_magnifications)
    
    # Calculate Delta RMS of Image Positions correctly
    squared_distances = 0.0
    for observed, predicted, _ in paired_positions:
        distance = calculate_distance(*observed, *predicted)
        squared_distances += distance ** 2  # Summing squared distances
    
    mean_squared_distances = squared_distances / len(paired_positions)
    delta_rms = np.sqrt(mean_squared_distances)  # Taking square root of the mean of squared distances
    delta_rms *= 1000  # Convert to mas
    
    # Order predicted positions and magnifications based on observed order
    predicted_positions_ordered = [pair[1] for pair in paired_positions]
    predicted_magnifications_ordered = [pair[2] for pair in paired_positions]
    
    # Determine the index of the "Green Image"
    green_index = 0  # CHOOSE WHICH IMAGE TO NORMALISE BY HERE !!!!!!!!!!!!!!!!!!!!!!!

    # Predicted Flux Ratios
    predicted_flux_ratios = [m / predicted_magnifications_ordered[green_index] for m in predicted_magnifications_ordered]

    with fits.open(fits_file) as hdul:
        mag = hdul[0].data[6]
    
    pixel_coords = [(int((x + mag.shape[1] // 2 * n) / n), int((y + mag.shape[0] // 2 * n) / n)) for x, y in observed_positions]
    values = [mag[y, x] for x, y in pixel_coords]
    
    abs_values = [1 / abs(value) for value in values]
    predicted_flux_ratios_observed = [abs_values[i] / abs_values[green_index] for i in range(len(abs_values))]

    # Calculate RMS excluding the green image
    squared_diffs_pred = [(pred - obs) ** 2 for i, (pred, obs) in enumerate(zip(predicted_flux_ratios, observed_flux_ratios)) if i != green_index]
    predicted_flux_ratios_rms = np.sqrt(sum(squared_diffs_pred) / len(squared_diffs_pred))

    squared_diffs_obs = [(pred - obs) ** 2 for i, (pred, obs) in enumerate(zip(predicted_flux_ratios_observed, observed_flux_ratios)) if i != green_index]
    predicted_flux_ratios_observed_rms = np.sqrt(sum(squared_diffs_obs) / len(squared_diffs_obs))

    return {
        'delta_rms': delta_rms,
        'predicted_flux_ratios_rms': predicted_flux_ratios_rms,
        'predicted_flux_ratios_observed_rms': predicted_flux_ratios_observed_rms
    }

# Function to extract chi^2 and lens models from optresult.dat file
def extract_chi2_and_lens_profiles(optresult_path):
    chi2_value = None
    profiles_used = []
    sie_count = 0

    with open(optresult_path, 'r') as file:
        lines = file.readlines()

    found_chi2 = False
    for line in lines:
        if line.startswith("chi^2"):
            chi2_value = float(line.split()[2])
            found_chi2 = True
            profiles_used = []
            sie_count = 0
        elif found_chi2:
            if line.startswith("lens   sie"):
                profiles_used.append("SIE")
                sie_count += 1
            elif line.startswith("lens   pow"):
                profiles_used.append("POW")
            elif line.startswith("lens   anfw"):
                profiles_used.append("NFW")
            elif line.startswith("lens   pert"):
                profiles_used.append("Shear")

    if chi2_value is None:
        raise ValueError("No chi^2 value found in the file.")

    profiles_used = list(set(profiles_used))
    return chi2_value, profiles_used, sie_count

# Function to calculate free parameters based on lens profiles
def calculate_free_parameters(profiles_used, sie_count):
    profile_params = {
        "SIE": 5,
        "POW": 6,
        "NFW": 6,
        "Shear": 5
    }
    total_params = sum(profile_params[profile] for profile in profiles_used)
    if sie_count == 2:
        total_params += 5
    return total_params

# Function to calculate BIC (Bayesian Information Criterion)
def calculate_bic(chi2, n_data_points, n_free_params):
    return chi2 + n_free_params * np.log(n_data_points)

# Function to compute BIC from optresult file
def compute_bic(optresult_path):
    chi2_value, profiles_used, sie_count = extract_chi2_and_lens_profiles(optresult_path)
    n_free_params = calculate_free_parameters(profiles_used, sie_count)
    
    # Adjust n_data_points based on "PF" in the filename
    if "PF" in optresult_path:
        n_data_points = 4 * 2 + 3
    else:
        n_data_points = 4 * 2
    
    bic_value = calculate_bic(chi2_value, n_data_points, n_free_params)
    return bic_value

# Main function to process multiple files
def process_files(input_file, output_files, fits_files, optresult_files, n):
    results = []
    latex_rows = []
    bics = []  # Store BIC values separately

    for output_file, fits_file, optresult_file in zip(output_files, fits_files, optresult_files):
        # Process output and fits files
        result = process_file_group(output_file, fits_file, input_file, n)
        results.append(result)
        
        # Process optresult files separately for BIC calculation
        bic = compute_bic(optresult_file)
        bics.append(bic)
        
        # Create LaTeX row
        model_name, constraint_name = determine_model_and_constraint(output_file)
        latex_row = f"{model_name} & {constraint_name} & {result['delta_rms']:.1f} & {result['predicted_flux_ratios_observed_rms']:.2f} & {result['predicted_flux_ratios_rms']:.2f} & {bic:.1f} \\\\"
        latex_rows.append(latex_row)
    
    # Combine results into a DataFrame
    df_results = pd.DataFrame(results)
    df_results['bic'] = bics  # Add BIC values to the DataFrame
    
    return df_results, latex_rows

# Determine model and constraint names from the filename
def determine_model_and_constraint(file_name):
    model_map = {'S': 'SIE', 'P': 'POW', 'N': 'NFW'}
    
    base_name = os.path.basename(file_name)
    model_key = base_name[3]
    constraint_key = base_name[4:6] if base_name[4:6] in ['PF'] else base_name[4]
    
    model_name = model_map.get(model_key, '')
    if 'R' in base_name:
        model_name += ' + Shear'
    if 'G' in base_name:
        model_name += ' + G2'
    
    constraint_name = 'QSO Pos + FR' if constraint_key == 'PF' else 'QSO Pos'
    
    return model_name, constraint_name


# Example usage with filenames
Input_file = "WFI2033/obs_point.dat"

Output_files = [
    "WFI2033/SP/outSP_point.dat",
    "WFI2033/SPR/outSPR_point.dat",
    "WFI2033/SPG/outSPG_point.dat",
    "WFI2033/SPGR/outSPGR_point.dat",
    "WFI2033/SPF/outSPF_point.dat",
    "WFI2033/SPFR/outSPFR_point.dat",
    "WFI2033/SPFG/outSPFG_point.dat",
    "WFI2033/SPFGR/outSPFGR_point.dat",
    "WFI2033/PP/outPP_point.dat",
    "WFI2033/PPR/outPPR_point.dat",
    "WFI2033/PPG/outPPG_point.dat",
    "WFI2033/PPGR/outPPGR_point.dat",
    "WFI2033/PPF/outPPF_point.dat",
    "WFI2033/PPFR/outPPFR_point.dat",
    "WFI2033/PPFG/outPPFG_point.dat",
    "WFI2033/PPFGR/outPPFGR_point.dat",
    "WFI2033/NP/outNP_point.dat",
    "WFI2033/NPR/outNPR_point.dat",
    "WFI2033/NPG/outNPG_point.dat",
    "WFI2033/NPGR/outNPGR_point.dat",
    "WFI2033/NPF/outNPF_point.dat",
    "WFI2033/NPFR/outNPFR_point.dat",
    "WFI2033/NPFG/outNPFG_point.dat",
    "WFI2033/NPFGR/outNPFGR_point.dat"
]

Fits_files = [
    "WFI2033/SP/outSP_lens.fits",
    "WFI2033/SPR/outSPR_lens.fits",
    "WFI2033/SPG/outSPG_lens.fits",
    "WFI2033/SPGR/outSPGR_lens.fits",
    "WFI2033/SPF/outSPF_lens.fits",
    "WFI2033/SPFR/outSPFR_lens.fits",
    "WFI2033/SPFG/outSPFG_lens.fits",
    "WFI2033/SPFGR/outSPFGR_lens.fits",
    "WFI2033/PP/outPP_lens.fits",
    "WFI2033/PPR/outPPR_lens.fits",
    "WFI2033/PPG/outPPG_lens.fits",
    "WFI2033/PPGR/outPPGR_lens.fits",
    "WFI2033/PPF/outPPF_lens.fits",
    "WFI2033/PPFR/outPPFR_lens.fits",
    "WFI2033/PPFG/outPPFG_lens.fits",
    "WFI2033/PPFGR/outPPFGR_lens.fits",
    "WFI2033/NP/outNP_lens.fits",
    "WFI2033/NPR/outNPR_lens.fits",
    "WFI2033/NPG/outNPG_lens.fits",
    "WFI2033/NPGR/outNPGR_lens.fits",
    "WFI2033/NPF/outNPF_lens.fits",
    "WFI2033/NPFR/outNPFR_lens.fits",
    "WFI2033/NPFG/outNPFG_lens.fits",
    "WFI2033/NPFGR/outNPFGR_lens.fits"
]

Optresult_files = [
    "WFI2033/SP/outSP_optresult.dat",
    "WFI2033/SPR/outSPR_optresult.dat",
    "WFI2033/SPG/outSPG_optresult.dat",
    "WFI2033/SPGR/outSPGR_optresult.dat",
    "WFI2033/SPF/outSPF_optresult.dat",
    "WFI2033/SPFR/outSPFR_optresult.dat",
    "WFI2033/SPFG/outSPFG_optresult.dat",
    "WFI2033/SPFGR/outSPFGR_optresult.dat",
    "WFI2033/PP/outPP_optresult.dat",
    "WFI2033/PPR/outPPR_optresult.dat",
    "WFI2033/PPG/outPPG_optresult.dat",
    "WFI2033/PPGR/outPPGR_optresult.dat",
    "WFI2033/PPF/outPPF_optresult.dat",
    "WFI2033/PPFR/outPPFR_optresult.dat",
    "WFI2033/PPFG/outPPFG_optresult.dat",
    "WFI2033/PPFGR/outPPFGR_optresult.dat",
    "WFI2033/NP/outNP_optresult.dat",
    "WFI2033/NPR/outNPR_optresult.dat",
    "WFI2033/NPG/outNPG_optresult.dat",
    "WFI2033/NPGR/outNPGR_optresult.dat",
    "WFI2033/NPF/outNPF_optresult.dat",
    "WFI2033/NPFR/outNPFR_optresult.dat",
    "WFI2033/NPFG/outNPFG_optresult.dat",
    "WFI2033/NPFGR/outNPFGR_optresult.dat"
]

n = 0.0025  # Example scale factor
results_df, latex_rows = process_files(Input_file, Output_files, Fits_files, Optresult_files, n)

# Print formatted results
for i, output_file in enumerate(Output_files):
    print(f"Model: {output_file}")
    print(f"Delta RMS of Image Positions: {results_df.iloc[i]['delta_rms']:.3f}")
    print(f"Delta RMS of flux ratios at the observed position: {results_df.iloc[i]['predicted_flux_ratios_observed_rms']:.2f}")
    print(f"Delta RMS of flux ratios at the predicted position: {results_df.iloc[i]['predicted_flux_ratios_rms']:.2f}")
    print(f"BIC: {results_df.iloc[i]['bic']:.3f}\n")

# Print LaTeX-formatted rows
print("LaTeX Table Rows:")
for row in latex_rows:
    print(row)

Model: WFI2033/SP/outSP_point.dat
Delta RMS of Image Positions: 49.571
Delta RMS of flux ratios at the observed position: 0.19
Delta RMS of flux ratios at the predicted position: 0.20
BIC: 511.740

Model: WFI2033/SPR/outSPR_point.dat
Delta RMS of Image Positions: 2.746
Delta RMS of flux ratios at the observed position: 0.23
Delta RMS of flux ratios at the predicted position: 0.22
BIC: 37.625

Model: WFI2033/SPG/outSPG_point.dat
Delta RMS of Image Positions: 12.764
Delta RMS of flux ratios at the observed position: 0.13
Delta RMS of flux ratios at the predicted position: 0.13
BIC: 30.371

Model: WFI2033/SPGR/outSPGR_point.dat
Delta RMS of Image Positions: 2.572
Delta RMS of flux ratios at the observed position: 0.14
Delta RMS of flux ratios at the predicted position: 0.14
BIC: 31.705

Model: WFI2033/SPF/outSPF_point.dat
Delta RMS of Image Positions: 50.794
Delta RMS of flux ratios at the observed position: 0.17
Delta RMS of flux ratios at the predicted position: 0.19
BIC: 336.843

Model

In [20]:
# USE THIS CELL, IT HAS THE FIXED LATEX AUTOMATED TABLE CODE 
# SDSSJ1330
import numpy as np
import pandas as pd
from astropy.io import fits
import os

# Function to calculate distance
def process_file_group(output_file, fits_file, input_file, n):
    observed_positions, observed_flux_ratios = read_observed_positions(input_file)
    predicted_positions, predicted_magnifications = read_predicted_positions(output_file)
    
    paired_positions = pair_positions(observed_positions, predicted_positions, predicted_magnifications)
    
    # Calculate Delta RMS of Image Positions correctly
    squared_distances = 0.0
    for observed, predicted, _ in paired_positions:
        distance = calculate_distance(*observed, *predicted)
        squared_distances += distance ** 2  # Summing squared distances
    
    mean_squared_distances = squared_distances / len(paired_positions)
    delta_rms = np.sqrt(mean_squared_distances)  # Taking square root of the mean of squared distances
    delta_rms *= 1000  # Convert to mas
    
    # Order predicted positions and magnifications based on observed order
    predicted_positions_ordered = [pair[1] for pair in paired_positions]
    predicted_magnifications_ordered = [pair[2] for pair in paired_positions]
    
    # Determine the index of the "Green Image"
    green_index = 0  # CHOOSE WHICH IMAGE TO NORMALISE BY HERE !!!!!!!!!!!!!!!!!!!!!!!

    # Predicted Flux Ratios
    predicted_flux_ratios = [m / predicted_magnifications_ordered[green_index] for m in predicted_magnifications_ordered]

    with fits.open(fits_file) as hdul:
        mag = hdul[0].data[6]
    
    pixel_coords = [(int((x + mag.shape[1] // 2 * n) / n), int((y + mag.shape[0] // 2 * n) / n)) for x, y in observed_positions]
    values = [mag[y, x] for x, y in pixel_coords]
    
    abs_values = [1 / abs(value) for value in values]
    predicted_flux_ratios_observed = [abs_values[i] / abs_values[green_index] for i in range(len(abs_values))]

    # Calculate RMS excluding the green image
    squared_diffs_pred = [(pred - obs) ** 2 for i, (pred, obs) in enumerate(zip(predicted_flux_ratios, observed_flux_ratios)) if i != green_index]
    predicted_flux_ratios_rms = np.sqrt(sum(squared_diffs_pred) / len(squared_diffs_pred))

    squared_diffs_obs = [(pred - obs) ** 2 for i, (pred, obs) in enumerate(zip(predicted_flux_ratios_observed, observed_flux_ratios)) if i != green_index]
    predicted_flux_ratios_observed_rms = np.sqrt(sum(squared_diffs_obs) / len(squared_diffs_obs))

    return {
        'delta_rms': delta_rms,
        'predicted_flux_ratios_rms': predicted_flux_ratios_rms,
        'predicted_flux_ratios_observed_rms': predicted_flux_ratios_observed_rms
    }

# Function to extract chi^2 and lens models from optresult.dat file
def extract_chi2_and_lens_profiles(optresult_path):
    chi2_value = None
    profiles_used = []
    sie_count = 0

    with open(optresult_path, 'r') as file:
        lines = file.readlines()

    found_chi2 = False
    for line in lines:
        if line.startswith("chi^2"):
            chi2_value = float(line.split()[2])
            found_chi2 = True
            profiles_used = []
            sie_count = 0
        elif found_chi2:
            if line.startswith("lens   sie"):
                profiles_used.append("SIE")
                sie_count += 1
            elif line.startswith("lens   pow"):
                profiles_used.append("POW")
            elif line.startswith("lens   anfw"):
                profiles_used.append("NFW")
            elif line.startswith("lens   pert"):
                profiles_used.append("Shear")

    if chi2_value is None:
        raise ValueError("No chi^2 value found in the file.")

    profiles_used = list(set(profiles_used))
    return chi2_value, profiles_used, sie_count

# Function to calculate free parameters based on lens profiles
def calculate_free_parameters(profiles_used, sie_count):
    profile_params = {
        "SIE": 5,
        "POW": 6,
        "NFW": 6,
        "Shear": 5
    }
    total_params = sum(profile_params[profile] for profile in profiles_used)
    if sie_count == 2:
        total_params += 5
    return total_params

# Function to calculate BIC (Bayesian Information Criterion)
def calculate_bic(chi2, n_data_points, n_free_params):
    return chi2 + n_free_params * np.log(n_data_points)

# Function to compute BIC from optresult file
def compute_bic(optresult_path):
    chi2_value, profiles_used, sie_count = extract_chi2_and_lens_profiles(optresult_path)
    n_free_params = calculate_free_parameters(profiles_used, sie_count)
    
    # Adjust n_data_points based on "PF" in the filename
    if "PF" in optresult_path:
        n_data_points = 4 * 2 + 3
    else:
        n_data_points = 4 * 2
    
    bic_value = calculate_bic(chi2_value, n_data_points, n_free_params)
    return bic_value

# Main function to process multiple files
def process_files(input_file, output_files, fits_files, optresult_files, n):
    results = []
    latex_rows = []
    bics = []  # Store BIC values separately

    for output_file, fits_file, optresult_file in zip(output_files, fits_files, optresult_files):
        # Process output and fits files
        result = process_file_group(output_file, fits_file, input_file, n)
        results.append(result)
        
        # Process optresult files separately for BIC calculation
        bic = compute_bic(optresult_file)
        bics.append(bic)
        
        # Create LaTeX row
        model_name, constraint_name = determine_model_and_constraint(output_file)
        latex_row = f"{model_name} & {constraint_name} & {result['delta_rms']:.1f} & {result['predicted_flux_ratios_observed_rms']:.2f} & {result['predicted_flux_ratios_rms']:.2f} & {bic:.1f} \\\\"
        latex_rows.append(latex_row)
    
    # Combine results into a DataFrame
    df_results = pd.DataFrame(results)
    df_results['bic'] = bics  # Add BIC values to the DataFrame
    
    return df_results, latex_rows

# Determine model and constraint names from the filename
def determine_model_and_constraint(file_name):
    model_map = {'S': 'SIE', 'P': 'POW', 'N': 'NFW'}
    
    base_name = os.path.basename(file_name)
    model_key = base_name[3]
    constraint_key = base_name[4:6] if base_name[4:6] in ['PF'] else base_name[4]
    
    model_name = model_map.get(model_key, '')
    if 'R' in base_name:
        model_name += ' + Shear'
    if 'G' in base_name:
        model_name += ' + G2'
    
    constraint_name = 'QSO Pos + FR' if constraint_key == 'PF' else 'QSO Pos'
    
    return model_name, constraint_name

# Example usage with filenames
Input_file = "SDSSJ1330/SPFC/Robs_point.dat"

Output_files = [
    "SDSSJ1330/SPC/outSP_point.dat",
    "SDSSJ1330/SPC/outSPR_point.dat",
    "SDSSJ1330/SPFC/outSPF_point.dat",
    "SDSSJ1330/SPFC/outSPFR_point.dat",
    "SDSSJ1330/PPC/outPP_point.dat",
    "SDSSJ1330/PPC/outPPR_point.dat",
    "SDSSJ1330/PPFC/outPPF_point.dat",
    "SDSSJ1330/PPFC/outPPFR_point.dat",
    "SDSSJ1330/NPC/outNP_point.dat",
    "SDSSJ1330/NPC/outNPR_point.dat",
    "SDSSJ1330/NPFC/outNPF_point.dat",
    "SDSSJ1330/NPFC/outNPFR_point.dat",
]

Fits_files = [
    "SDSSJ1330/SPC/outSP_lens.fits",
    "SDSSJ1330/SPC/outSPR_lens.fits",
    "SDSSJ1330/SPFC/outSPF_lens.fits",
    "SDSSJ1330/SPFC/outSPFR_lens.fits",
    "SDSSJ1330/PPC/outPP_lens.fits",
    "SDSSJ1330/PPC/outPPR_lens.fits",
    "SDSSJ1330/PPFC/outPPF_lens.fits",
    "SDSSJ1330/PPFC/outPPFR_lens.fits",
    "SDSSJ1330/NPC/outNP_lens.fits",
    "SDSSJ1330/NPC/outNPR_lens.fits",
    "SDSSJ1330/NPFC/outNPF_lens.fits",
    "SDSSJ1330/NPFC/outNPFR_lens.fits",
]

Optresult_files = [
    "SDSSJ1330/SPC/outSP_optresult.dat",
    "SDSSJ1330/SPC/outSPR_optresult.dat",
    "SDSSJ1330/SPFC/outSPF_optresult.dat",
    "SDSSJ1330/SPFC/outSPFR_optresult.dat",
    "SDSSJ1330/PPC/outPP_optresult.dat",
    "SDSSJ1330/PPC/outPPR_optresult.dat",
    "SDSSJ1330/PPFC/outPPF_optresult.dat",
    "SDSSJ1330/PPFC/outPPFR_optresult.dat",
    "SDSSJ1330/NPC/outNP_optresult.dat",
    "SDSSJ1330/NPC/outNPR_optresult.dat",
    "SDSSJ1330/NPFC/outNPF_optresult.dat",
    "SDSSJ1330/NPFC/outNPFR_optresult.dat",
]

n = 0.0025  # Example scale factor
results_df, latex_rows = process_files(Input_file, Output_files, Fits_files, Optresult_files, n)

# Print formatted results
for i, output_file in enumerate(Output_files):
    print(f"Model: {output_file}")
    print(f"Delta RMS of Image Positions: {results_df.iloc[i]['delta_rms']:.3f}")
    print(f"Delta RMS of flux ratios at the observed position: {results_df.iloc[i]['predicted_flux_ratios_observed_rms']:.2f}")
    print(f"Delta RMS of flux ratios at the predicted position: {results_df.iloc[i]['predicted_flux_ratios_rms']:.2f}")
    print(f"BIC: {results_df.iloc[i]['bic']:.3f}\n")

# Print LaTeX-formatted rows
print("LaTeX Table Rows:")
for row in latex_rows:
    print(row)

Model: SDSSJ1330/SPC/outSP_point.dat
Delta RMS of Image Positions: 32.203
Delta RMS of flux ratios at the observed position: 0.65
Delta RMS of flux ratios at the predicted position: 0.12
BIC: 183.888

Model: SDSSJ1330/SPC/outSPR_point.dat
Delta RMS of Image Positions: 0.887
Delta RMS of flux ratios at the observed position: 0.08
Delta RMS of flux ratios at the predicted position: 0.08
BIC: 22.021

Model: SDSSJ1330/SPFC/outSPF_point.dat
Delta RMS of Image Positions: 32.222
Delta RMS of flux ratios at the observed position: 0.63
Delta RMS of flux ratios at the predicted position: 0.12
BIC: 210.202

Model: SDSSJ1330/SPFC/outSPFR_point.dat
Delta RMS of Image Positions: 3.112
Delta RMS of flux ratios at the observed position: 0.06
Delta RMS of flux ratios at the predicted position: 0.06
BIC: 31.973

Model: SDSSJ1330/PPC/outPP_point.dat
Delta RMS of Image Positions: 32.107
Delta RMS of flux ratios at the observed position: 1.18
Delta RMS of flux ratios at the predicted position: 0.16
BIC: 18

In [21]:
# USE THIS CELL, IT HAS THE FIXED LATEX AUTOMATED TABLE CODE 
# WFI2026
import numpy as np
import pandas as pd
from astropy.io import fits
import os

# Function to calculate distance
def process_file_group(output_file, fits_file, input_file, n):
    observed_positions, observed_flux_ratios = read_observed_positions(input_file)
    predicted_positions, predicted_magnifications = read_predicted_positions(output_file)
    
    paired_positions = pair_positions(observed_positions, predicted_positions, predicted_magnifications)
    
    # Calculate Delta RMS of Image Positions correctly
    squared_distances = 0.0
    for observed, predicted, _ in paired_positions:
        distance = calculate_distance(*observed, *predicted)
        squared_distances += distance ** 2  # Summing squared distances
    
    mean_squared_distances = squared_distances / len(paired_positions)
    delta_rms = np.sqrt(mean_squared_distances)  # Taking square root of the mean of squared distances
    delta_rms *= 1000  # Convert to mas
    
    # Order predicted positions and magnifications based on observed order
    predicted_positions_ordered = [pair[1] for pair in paired_positions]
    predicted_magnifications_ordered = [pair[2] for pair in paired_positions]
    
    # Determine the index of the "Green Image"
    green_index = 0  # CHOOSE WHICH IMAGE TO NORMALISE BY HERE !!!!!!!!!!!!!!!!!!!!!!!

    # Predicted Flux Ratios
    predicted_flux_ratios = [m / predicted_magnifications_ordered[green_index] for m in predicted_magnifications_ordered]

    with fits.open(fits_file) as hdul:
        mag = hdul[0].data[6]
    
    pixel_coords = [(int((x + mag.shape[1] // 2 * n) / n), int((y + mag.shape[0] // 2 * n) / n)) for x, y in observed_positions]
    values = [mag[y, x] for x, y in pixel_coords]
    
    abs_values = [1 / abs(value) for value in values]
    predicted_flux_ratios_observed = [abs_values[i] / abs_values[green_index] for i in range(len(abs_values))]

    # Calculate RMS excluding the green image
    squared_diffs_pred = [(pred - obs) ** 2 for i, (pred, obs) in enumerate(zip(predicted_flux_ratios, observed_flux_ratios)) if i != green_index]
    predicted_flux_ratios_rms = np.sqrt(sum(squared_diffs_pred) / len(squared_diffs_pred))

    squared_diffs_obs = [(pred - obs) ** 2 for i, (pred, obs) in enumerate(zip(predicted_flux_ratios_observed, observed_flux_ratios)) if i != green_index]
    predicted_flux_ratios_observed_rms = np.sqrt(sum(squared_diffs_obs) / len(squared_diffs_obs))

    return {
        'delta_rms': delta_rms,
        'predicted_flux_ratios_rms': predicted_flux_ratios_rms,
        'predicted_flux_ratios_observed_rms': predicted_flux_ratios_observed_rms
    }

# Function to extract chi^2 and lens models from optresult.dat file
def extract_chi2_and_lens_profiles(optresult_path):
    chi2_value = None
    profiles_used = []
    sie_count = 0

    with open(optresult_path, 'r') as file:
        lines = file.readlines()

    found_chi2 = False
    for line in lines:
        if line.startswith("chi^2"):
            chi2_value = float(line.split()[2])
            found_chi2 = True
            profiles_used = []
            sie_count = 0
        elif found_chi2:
            if line.startswith("lens   sie"):
                profiles_used.append("SIE")
                sie_count += 1
            elif line.startswith("lens   pow"):
                profiles_used.append("POW")
            elif line.startswith("lens   anfw"):
                profiles_used.append("NFW")
            elif line.startswith("lens   pert"):
                profiles_used.append("Shear")

    if chi2_value is None:
        raise ValueError("No chi^2 value found in the file.")

    profiles_used = list(set(profiles_used))
    return chi2_value, profiles_used, sie_count

# Function to calculate free parameters based on lens profiles
def calculate_free_parameters(profiles_used, sie_count):
    profile_params = {
        "SIE": 5,
        "POW": 6,
        "NFW": 6,
        "Shear": 5
    }
    total_params = sum(profile_params[profile] for profile in profiles_used)
    if sie_count == 2:
        total_params += 5
    return total_params

# Function to calculate BIC (Bayesian Information Criterion)
def calculate_bic(chi2, n_data_points, n_free_params):
    return chi2 + n_free_params * np.log(n_data_points)

# Function to compute BIC from optresult file
def compute_bic(optresult_path):
    chi2_value, profiles_used, sie_count = extract_chi2_and_lens_profiles(optresult_path)
    n_free_params = calculate_free_parameters(profiles_used, sie_count)
    
    # Adjust n_data_points based on "PF" in the filename
    if "PF" in optresult_path:
        n_data_points = 4 * 2 + 3
    else:
        n_data_points = 4 * 2
    
    bic_value = calculate_bic(chi2_value, n_data_points, n_free_params)
    return bic_value

# Main function to process multiple files
def process_files(input_file, output_files, fits_files, optresult_files, n):
    results = []
    latex_rows = []
    bics = []  # Store BIC values separately

    for output_file, fits_file, optresult_file in zip(output_files, fits_files, optresult_files):
        # Process output and fits files
        result = process_file_group(output_file, fits_file, input_file, n)
        results.append(result)
        
        # Process optresult files separately for BIC calculation
        bic = compute_bic(optresult_file)
        bics.append(bic)
        
        # Create LaTeX row
        model_name, constraint_name = determine_model_and_constraint(output_file)
        latex_row = f"{model_name} & {constraint_name} & {result['delta_rms']:.1f} & {result['predicted_flux_ratios_observed_rms']:.2f} & {result['predicted_flux_ratios_rms']:.2f} & {bic:.1f} \\\\"
        latex_rows.append(latex_row)
    
    # Combine results into a DataFrame
    df_results = pd.DataFrame(results)
    df_results['bic'] = bics  # Add BIC values to the DataFrame
    
    return df_results, latex_rows

# Determine model and constraint names from the filename
def determine_model_and_constraint(file_name):
    model_map = {'S': 'SIE', 'P': 'POW', 'N': 'NFW'}
    
    base_name = os.path.basename(file_name)
    model_key = base_name[3]
    constraint_key = base_name[4:6] if base_name[4:6] in ['PF'] else base_name[4]
    
    model_name = model_map.get(model_key, '')
    if 'R' in base_name:
        model_name += ' + Shear'
    if 'G' in base_name:
        model_name += ' + G2'
    
    constraint_name = 'QSO Pos + FR' if constraint_key == 'PF' else 'QSO Pos'
    
    return model_name, constraint_name

# Example usage with filenames
Input_file = "WFI2026/SPFC/Wobs_point.dat"

Output_files = [
    "WFI2026/SPC/outSP_point.dat",
    "WFI2026/SPC/outSPR_point.dat",
    "WFI2026/SPFC/outSPF_point.dat",
    "WFI2026/SPFC/outSPFR_point.dat",
    "WFI2026/PPC/outPP_point.dat",
    "WFI2026/PPC/outPPR_point.dat",
    "WFI2026/PPFC/outPPF_point.dat",
    "WFI2026/PPFC/outPPFR_point.dat",
    "WFI2026/NPC/outNP_point.dat",
    "WFI2026/NPC/outNPR_point.dat",
    "WFI2026/NPFC/outNPF_point.dat",
    "WFI2026/NPFC/outNPFR_point.dat",
]

Fits_files = [
    "WFI2026/SPC/outSP_lens.fits",
    "WFI2026/SPC/outSPR_lens.fits",
    "WFI2026/SPFC/outSPF_lens.fits",
    "WFI2026/SPFC/outSPFR_lens.fits",
    "WFI2026/PPC/outPP_lens.fits",
    "WFI2026/PPC/outPPR_lens.fits",
    "WFI2026/PPFC/outPPF_lens.fits",
    "WFI2026/PPFC/outPPFR_lens.fits",
    "WFI2026/NPC/outNP_lens.fits",
    "WFI2026/NPC/outNPR_lens.fits",
    "WFI2026/NPFC/outNPF_lens.fits",
    "WFI2026/NPFC/outNPFR_lens.fits",
]

Optresult_files = [
    "WFI2026/SPC/outSP_optresult.dat",
    "WFI2026/SPC/outSPR_optresult.dat",
    "WFI2026/SPFC/outSPF_optresult.dat",
    "WFI2026/SPFC/outSPFR_optresult.dat",
    "WFI2026/PPC/outPP_optresult.dat",
    "WFI2026/PPC/outPPR_optresult.dat",
    "WFI2026/PPFC/outPPF_optresult.dat",
    "WFI2026/PPFC/outPPFR_optresult.dat",
    "WFI2026/NPC/outNP_optresult.dat",
    "WFI2026/NPC/outNPR_optresult.dat",
    "WFI2026/NPFC/outNPF_optresult.dat",
    "WFI2026/NPFC/outNPFR_optresult.dat",
]

n = 0.0025  # Example scale factor
results_df, latex_rows = process_files(Input_file, Output_files, Fits_files, Optresult_files, n)

# Print formatted results
for i, output_file in enumerate(Output_files):
    print(f"Model: {output_file}")
    print(f"Delta RMS of Image Positions: {results_df.iloc[i]['delta_rms']:.3f}")
    print(f"Delta RMS of flux ratios at the observed position: {results_df.iloc[i]['predicted_flux_ratios_observed_rms']:.3f}")
    print(f"Delta RMS of flux ratios at the predicted position: {results_df.iloc[i]['predicted_flux_ratios_rms']:.3f}")
    print(f"BIC: {results_df.iloc[i]['bic']:.3f}\n")

# Print LaTeX-formatted rows
print("LaTeX Table Rows:")
for row in latex_rows:
    print(row)

Model: WFI2026/SPC/outSP_point.dat
Delta RMS of Image Positions: 15.162
Delta RMS of flux ratios at the observed position: 0.104
Delta RMS of flux ratios at the predicted position: 0.034
BIC: 78.432

Model: WFI2026/SPC/outSPR_point.dat
Delta RMS of Image Positions: 5.065
Delta RMS of flux ratios at the observed position: 0.042
Delta RMS of flux ratios at the predicted position: 0.034
BIC: 51.920

Model: WFI2026/SPFC/outSPF_point.dat
Delta RMS of Image Positions: 16.351
Delta RMS of flux ratios at the observed position: 0.107
Delta RMS of flux ratios at the predicted position: 0.030
BIC: 90.575

Model: WFI2026/SPFC/outSPFR_point.dat
Delta RMS of Image Positions: 3.992
Delta RMS of flux ratios at the observed position: 0.009
Delta RMS of flux ratios at the predicted position: 0.008
BIC: 58.103

Model: WFI2026/PPC/outPP_point.dat
Delta RMS of Image Positions: 11.098
Delta RMS of flux ratios at the observed position: 0.110
Delta RMS of flux ratios at the predicted position: 0.038
BIC: 68.6

In [22]:
# USE THIS CELL, IT HAS THE FIXED LATEX AUTOMATED TABLE CODE 
# WGDJ0405
import numpy as np
import pandas as pd
from astropy.io import fits
import os

# Function to calculate distance
def process_file_group(output_file, fits_file, input_file, n):
    observed_positions, observed_flux_ratios = read_observed_positions(input_file)
    predicted_positions, predicted_magnifications = read_predicted_positions(output_file)
    
    paired_positions = pair_positions(observed_positions, predicted_positions, predicted_magnifications)
    
    # Calculate Delta RMS of Image Positions correctly
    squared_distances = 0.0
    for observed, predicted, _ in paired_positions:
        distance = calculate_distance(*observed, *predicted)
        squared_distances += distance ** 2  # Summing squared distances
    
    mean_squared_distances = squared_distances / len(paired_positions)
    delta_rms = np.sqrt(mean_squared_distances)  # Taking square root of the mean of squared distances
    delta_rms *= 1000  # Convert to mas
    
    # Order predicted positions and magnifications based on observed order
    predicted_positions_ordered = [pair[1] for pair in paired_positions]
    predicted_magnifications_ordered = [pair[2] for pair in paired_positions]
    
    # Determine the index of the "Green Image"
    green_index = 0  # CHOOSE WHICH IMAGE TO NORMALISE BY HERE !!!!!!!!!!!!!!!!!!!!!!!

    # Predicted Flux Ratios
    predicted_flux_ratios = [m / predicted_magnifications_ordered[green_index] for m in predicted_magnifications_ordered]

    with fits.open(fits_file) as hdul:
        mag = hdul[0].data[6]
    
    pixel_coords = [(int((x + mag.shape[1] // 2 * n) / n), int((y + mag.shape[0] // 2 * n) / n)) for x, y in observed_positions]
    values = [mag[y, x] for x, y in pixel_coords]
    
    abs_values = [1 / abs(value) for value in values]
    predicted_flux_ratios_observed = [abs_values[i] / abs_values[green_index] for i in range(len(abs_values))]

    # Calculate RMS excluding the green image
    squared_diffs_pred = [(pred - obs) ** 2 for i, (pred, obs) in enumerate(zip(predicted_flux_ratios, observed_flux_ratios)) if i != green_index]
    predicted_flux_ratios_rms = np.sqrt(sum(squared_diffs_pred) / len(squared_diffs_pred))

    squared_diffs_obs = [(pred - obs) ** 2 for i, (pred, obs) in enumerate(zip(predicted_flux_ratios_observed, observed_flux_ratios)) if i != green_index]
    predicted_flux_ratios_observed_rms = np.sqrt(sum(squared_diffs_obs) / len(squared_diffs_obs))

    return {
        'delta_rms': delta_rms,
        'predicted_flux_ratios_rms': predicted_flux_ratios_rms,
        'predicted_flux_ratios_observed_rms': predicted_flux_ratios_observed_rms
    }

# Function to extract chi^2 and lens models from optresult.dat file
def extract_chi2_and_lens_profiles(optresult_path):
    chi2_value = None
    profiles_used = []
    sie_count = 0

    with open(optresult_path, 'r') as file:
        lines = file.readlines()

    found_chi2 = False
    for line in lines:
        if line.startswith("chi^2"):
            chi2_value = float(line.split()[2])
            found_chi2 = True
            profiles_used = []
            sie_count = 0
        elif found_chi2:
            if line.startswith("lens   sie"):
                profiles_used.append("SIE")
                sie_count += 1
            elif line.startswith("lens   pow"):
                profiles_used.append("POW")
            elif line.startswith("lens   anfw"):
                profiles_used.append("NFW")
            elif line.startswith("lens   pert"):
                profiles_used.append("Shear")

    if chi2_value is None:
        raise ValueError("No chi^2 value found in the file.")

    profiles_used = list(set(profiles_used))
    return chi2_value, profiles_used, sie_count

# Function to calculate free parameters based on lens profiles
def calculate_free_parameters(profiles_used, sie_count):
    profile_params = {
        "SIE": 5,
        "POW": 6,
        "NFW": 6,
        "Shear": 5
    }
    total_params = sum(profile_params[profile] for profile in profiles_used)
    if sie_count == 2:
        total_params += 5
    return total_params

# Function to calculate BIC (Bayesian Information Criterion)
def calculate_bic(chi2, n_data_points, n_free_params):
    return chi2 + n_free_params * np.log(n_data_points)

# Function to compute BIC from optresult file
def compute_bic(optresult_path):
    chi2_value, profiles_used, sie_count = extract_chi2_and_lens_profiles(optresult_path)
    n_free_params = calculate_free_parameters(profiles_used, sie_count)
    
    # Adjust n_data_points based on "PF" in the filename
    if "PF" in optresult_path:
        n_data_points = 4 * 2 + 3
    else:
        n_data_points = 4 * 2
    
    bic_value = calculate_bic(chi2_value, n_data_points, n_free_params)
    return bic_value

# Main function to process multiple files
def process_files(input_file, output_files, fits_files, optresult_files, n):
    results = []
    latex_rows = []
    bics = []  # Store BIC values separately

    for output_file, fits_file, optresult_file in zip(output_files, fits_files, optresult_files):
        # Process output and fits files
        result = process_file_group(output_file, fits_file, input_file, n)
        results.append(result)
        
        # Process optresult files separately for BIC calculation
        bic = compute_bic(optresult_file)
        bics.append(bic)
        
        # Create LaTeX row
        model_name, constraint_name = determine_model_and_constraint(output_file)
        latex_row = f"{model_name} & {constraint_name} & {result['delta_rms']:.1f} & {result['predicted_flux_ratios_observed_rms']:.2f} & {result['predicted_flux_ratios_rms']:.2f} & {bic:.1f} \\\\"
        latex_rows.append(latex_row)
    
    # Combine results into a DataFrame
    df_results = pd.DataFrame(results)
    df_results['bic'] = bics  # Add BIC values to the DataFrame
    
    return df_results, latex_rows

# Determine model and constraint names from the filename
def determine_model_and_constraint(file_name):
    model_map = {'S': 'SIE', 'P': 'POW', 'N': 'NFW'}
    
    base_name = os.path.basename(file_name)
    model_key = base_name[3]
    constraint_key = base_name[4:6] if base_name[4:6] in ['PF'] else base_name[4]
    
    model_name = model_map.get(model_key, '')
    if 'R' in base_name:
        model_name += ' + Shear'
    if 'G' in base_name:
        model_name += ' + G2'
    
    constraint_name = 'QSO Pos + FR' if constraint_key == 'PF' else 'QSO Pos'
    
    return model_name, constraint_name

# Example usage with filenames
Input_file = "WGDJ0405/SPFC/Eobs_point.dat"

Output_files = [
    "WGDJ0405/SPC/outSP_point.dat",
    "WGDJ0405/SPC/outSPR_point.dat",
    "WGDJ0405/SPFC/outSPF_point.dat",
    "WGDJ0405/SPFC/outSPFR_point.dat",
    "WGDJ0405/PPC/outPP_point.dat",
    "WGDJ0405/PPC/outPPR_point.dat",
    "WGDJ0405/PPFC/outPPF_point.dat",
    "WGDJ0405/PPFC/outPPFR_point.dat",
    "WGDJ0405/NPC/outNP_point.dat",
    "WGDJ0405/NPC/outNPR_point.dat",
    "WGDJ0405/NPFC/outNPF_point.dat",
    "WGDJ0405/NPFC/outNPFR_point.dat"
]

Fits_files = [
    "WGDJ0405/SPC/outSP_lens.fits",
    "WGDJ0405/SPC/outSPR_lens.fits",
    "WGDJ0405/SPFC/outSPF_lens.fits",
    "WGDJ0405/SPFC/outSPFR_lens.fits",
    "WGDJ0405/PPC/outPP_lens.fits",
    "WGDJ0405/PPC/outPPR_lens.fits",
    "WGDJ0405/PPFC/outPPF_lens.fits",
    "WGDJ0405/PPFC/outPPFR_lens.fits",
    "WGDJ0405/NPC/outNP_lens.fits",
    "WGDJ0405/NPC/outNPR_lens.fits",
    "WGDJ0405/NPFC/outNPF_lens.fits",
    "WGDJ0405/NPFC/outNPFR_lens.fits"
]

Optresult_files = [
    "WGDJ0405/SPC/outSP_optresult.dat",
    "WGDJ0405/SPC/outSPR_optresult.dat",
    "WGDJ0405/SPFC/outSPF_optresult.dat",
    "WGDJ0405/SPFC/outSPFR_optresult.dat",
    "WGDJ0405/PPC/outPP_optresult.dat",
    "WGDJ0405/PPC/outPPR_optresult.dat",
    "WGDJ0405/PPFC/outPPF_optresult.dat",
    "WGDJ0405/PPFC/outPPFR_optresult.dat",
    "WGDJ0405/NPC/outNP_optresult.dat",
    "WGDJ0405/NPC/outNPR_optresult.dat",
    "WGDJ0405/NPFC/outNPF_optresult.dat",
    "WGDJ0405/NPFC/outNPFR_optresult.dat"
]

n = 0.0025  # Example scale factor
results_df, latex_rows = process_files(Input_file, Output_files, Fits_files, Optresult_files, n)

# Print formatted results
for i, output_file in enumerate(Output_files):
    print(f"Model: {output_file}")
    print(f"Delta RMS of Image Positions: {results_df.iloc[i]['delta_rms']:.3f}")
    print(f"Delta RMS of flux ratios at the observed position: {results_df.iloc[i]['predicted_flux_ratios_observed_rms']:.2f}")
    print(f"Delta RMS of flux ratios at the predicted position: {results_df.iloc[i]['predicted_flux_ratios_rms']:.2f}")
    print(f"BIC: {results_df.iloc[i]['bic']:.3f}\n")

# Print LaTeX-formatted rows
print("LaTeX Table Rows:")
for row in latex_rows:
    print(row)

Model: WGDJ0405/SPC/outSP_point.dat
Delta RMS of Image Positions: 17.938
Delta RMS of flux ratios at the observed position: 0.38
Delta RMS of flux ratios at the predicted position: 0.14
BIC: 65.373

Model: WGDJ0405/SPC/outSPR_point.dat
Delta RMS of Image Positions: 1.655
Delta RMS of flux ratios at the observed position: 0.18
Delta RMS of flux ratios at the predicted position: 0.14
BIC: 25.625

Model: WGDJ0405/SPFC/outSPF_point.dat
Delta RMS of Image Positions: 18.027
Delta RMS of flux ratios at the observed position: 0.38
Delta RMS of flux ratios at the predicted position: 0.15
BIC: 81.427

Model: WGDJ0405/SPFC/outSPFR_point.dat
Delta RMS of Image Positions: 2.658
Delta RMS of flux ratios at the observed position: 0.26
Delta RMS of flux ratios at the predicted position: 0.21
BIC: 38.021

Model: WGDJ0405/PPC/outPP_point.dat
Delta RMS of Image Positions: 17.946
Delta RMS of flux ratios at the observed position: 0.54
Delta RMS of flux ratios at the predicted position: 0.19
BIC: 67.757

M

In [12]:
# USE THIS CELL, IT HAS THE FIXED LATEX AUTOMATED TABLE CODE 
# WGD2038
import numpy as np
import pandas as pd
from astropy.io import fits
import os

# Function to calculate distance
def process_file_group(output_file, fits_file, input_file, n):
    observed_positions, observed_flux_ratios = read_observed_positions(input_file)
    predicted_positions, predicted_magnifications = read_predicted_positions(output_file)
    
    paired_positions = pair_positions(observed_positions, predicted_positions, predicted_magnifications)
    
    # Calculate Delta RMS of Image Positions correctly
    squared_distances = 0.0
    for observed, predicted, _ in paired_positions:
        distance = calculate_distance(*observed, *predicted)
        squared_distances += distance ** 2  # Summing squared distances
    
    mean_squared_distances = squared_distances / len(paired_positions)
    delta_rms = np.sqrt(mean_squared_distances)  # Taking square root of the mean of squared distances
    delta_rms *= 1000  # Convert to mas
    
    # Order predicted positions and magnifications based on observed order
    predicted_positions_ordered = [pair[1] for pair in paired_positions]
    predicted_magnifications_ordered = [pair[2] for pair in paired_positions]
    
    # Determine the index of the "Green Image"
    green_index = 0  # CHOOSE WHICH IMAGE TO NORMALISE BY HERE !!!!!!!!!!!!!!!!!!!!!!!

    # Predicted Flux Ratios
    predicted_flux_ratios = [m / predicted_magnifications_ordered[green_index] for m in predicted_magnifications_ordered]

    with fits.open(fits_file) as hdul:
        mag = hdul[0].data[6]
    
    pixel_coords = [(int((x + mag.shape[1] // 2 * n) / n), int((y + mag.shape[0] // 2 * n) / n)) for x, y in observed_positions]
    values = [mag[y, x] for x, y in pixel_coords]
    
    abs_values = [1 / abs(value) for value in values]
    predicted_flux_ratios_observed = [abs_values[i] / abs_values[green_index] for i in range(len(abs_values))]

    # Calculate RMS excluding the green image
    squared_diffs_pred = [(pred - obs) ** 2 for i, (pred, obs) in enumerate(zip(predicted_flux_ratios, observed_flux_ratios)) if i != green_index]
    predicted_flux_ratios_rms = np.sqrt(sum(squared_diffs_pred) / len(squared_diffs_pred))

    squared_diffs_obs = [(pred - obs) ** 2 for i, (pred, obs) in enumerate(zip(predicted_flux_ratios_observed, observed_flux_ratios)) if i != green_index]
    predicted_flux_ratios_observed_rms = np.sqrt(sum(squared_diffs_obs) / len(squared_diffs_obs))

    return {
        'delta_rms': delta_rms,
        'predicted_flux_ratios_rms': predicted_flux_ratios_rms,
        'predicted_flux_ratios_observed_rms': predicted_flux_ratios_observed_rms
    }

# Function to extract chi^2 and lens models from optresult.dat file
def extract_chi2_and_lens_profiles(optresult_path):
    chi2_value = None
    profiles_used = []
    sie_count = 0

    with open(optresult_path, 'r') as file:
        lines = file.readlines()

    found_chi2 = False
    for line in lines:
        if line.startswith("chi^2"):
            chi2_value = float(line.split()[2])
            found_chi2 = True
            profiles_used = []
            sie_count = 0
        elif found_chi2:
            if line.startswith("lens   sie"):
                profiles_used.append("SIE")
                sie_count += 1
            elif line.startswith("lens   pow"):
                profiles_used.append("POW")
            elif line.startswith("lens   anfw"):
                profiles_used.append("NFW")
            elif line.startswith("lens   pert"):
                profiles_used.append("Shear")

    if chi2_value is None:
        raise ValueError("No chi^2 value found in the file.")

    profiles_used = list(set(profiles_used))
    return chi2_value, profiles_used, sie_count

# Function to calculate free parameters based on lens profiles
def calculate_free_parameters(profiles_used, sie_count):
    profile_params = {
        "SIE": 5,
        "POW": 6,
        "NFW": 6,
        "Shear": 5
    }
    total_params = sum(profile_params[profile] for profile in profiles_used)
    if sie_count == 2:
        total_params += 5
    return total_params

# Function to calculate BIC (Bayesian Information Criterion)
def calculate_bic(chi2, n_data_points, n_free_params):
    return chi2 + n_free_params * np.log(n_data_points)

# Function to compute BIC from optresult file
def compute_bic(optresult_path):
    chi2_value, profiles_used, sie_count = extract_chi2_and_lens_profiles(optresult_path)
    n_free_params = calculate_free_parameters(profiles_used, sie_count)
    
    # Adjust n_data_points based on "PF" in the filename
    if "PF" in optresult_path:
        n_data_points = 4 * 2 + 3
    else:
        n_data_points = 4 * 2
    
    bic_value = calculate_bic(chi2_value, n_data_points, n_free_params)
    return bic_value

# Main function to process multiple files
def process_files(input_file, output_files, fits_files, optresult_files, n):
    results = []
    latex_rows = []
    bics = []  # Store BIC values separately

    for output_file, fits_file, optresult_file in zip(output_files, fits_files, optresult_files):
        # Process output and fits files
        result = process_file_group(output_file, fits_file, input_file, n)
        results.append(result)
        
        # Process optresult files separately for BIC calculation
        bic = compute_bic(optresult_file)
        bics.append(bic)
        
        # Create LaTeX row
        model_name, constraint_name = determine_model_and_constraint(output_file)
        latex_row = f"{model_name} & {constraint_name} & {result['delta_rms']:.1f} & {result['predicted_flux_ratios_observed_rms']:.2f} & {result['predicted_flux_ratios_rms']:.2f} & {bic:.1f} \\\\"
        latex_rows.append(latex_row)
    
    # Combine results into a DataFrame
    df_results = pd.DataFrame(results)
    df_results['bic'] = bics  # Add BIC values to the DataFrame
    
    return df_results, latex_rows

# Determine model and constraint names from the filename
def determine_model_and_constraint(file_name):
    model_map = {'S': 'SIE', 'P': 'POW', 'N': 'NFW'}
    
    base_name = os.path.basename(file_name)
    model_key = base_name[3]
    constraint_key = base_name[4:6] if base_name[4:6] in ['PF'] else base_name[4]
    
    model_name = model_map.get(model_key, '')
    if 'R' in base_name:
        model_name += ' + Shear'
    if 'G' in base_name:
        model_name += ' + G2'
    
    constraint_name = 'QSO Pos + FR' if constraint_key == 'PF' else 'QSO Pos'
    
    return model_name, constraint_name

# Example usage with filenames
Input_file = "WGD2038/obs_point_SIE(POS+FLUX).dat"

Output_files = [
    "WGD2038/SPC/outSP_point.dat",
    "WGD2038/SPC/outSPR_point.dat",
    "WGD2038/SPFC/outSPF_point.dat",
    "WGD2038/SPFC/outSPFR_point.dat",
    "WGD2038/PPC/outPP_point.dat",
    "WGD2038/PPC/outPPR_point.dat",
    "WGD2038/PPFC/outPPF_point.dat",
    "WGD2038/PPFC/outPPFR_point.dat",
    "WGD2038/NPC/outNP_point.dat",
    "WGD2038/NPC/outNPR_point.dat",
    "WGD2038/NPFC/outNPF_point.dat",
    "WGD2038/NPFC/outNPFR_point.dat"
]

Fits_files = [
    "WGD2038/SPC/outSP_lens.fits",
    "WGD2038/SPC/outSPR_lens.fits",
    "WGD2038/SPFC/outSPF_lens.fits",
    "WGD2038/SPFC/outSPFR_lens.fits",
    "WGD2038/PPC/outPP_lens.fits",
    "WGD2038/PPC/outPPR_lens.fits",
    "WGD2038/PPFC/outPPF_lens.fits",
    "WGD2038/PPFC/outPPFR_lens.fits",
    "WGD2038/NPC/outNP_lens.fits",
    "WGD2038/NPC/outNPR_lens.fits",
    "WGD2038/NPFC/outNPF_lens.fits",
    "WGD2038/NPFC/outNPFR_lens.fits"
]

Optresult_files = [
    "WGD2038/SPC/outSP_optresult.dat",
    "WGD2038/SPC/outSPR_optresult.dat",
    "WGD2038/SPFC/outSPF_optresult.dat",
    "WGD2038/SPFC/outSPFR_optresult.dat",
    "WGD2038/PPC/outPP_optresult.dat",
    "WGD2038/PPC/outPPR_optresult.dat",
    "WGD2038/PPFC/outPPF_optresult.dat",
    "WGD2038/PPFC/outPPFR_optresult.dat",
    "WGD2038/NPC/outNP_optresult.dat",
    "WGD2038/NPC/outNPR_optresult.dat",
    "WGD2038/NPFC/outNPF_optresult.dat",
    "WGD2038/NPFC/outNPFR_optresult.dat"
]

n = 0.0025  # Example scale factor
results_df, latex_rows = process_files(Input_file, Output_files, Fits_files, Optresult_files, n)

# Print formatted results
for i, output_file in enumerate(Output_files):
    print(f"Model: {output_file}")
    print(f"Delta RMS of Image Positions: {results_df.iloc[i]['delta_rms']:.3f}")
    print(f"Delta RMS of flux ratios at the observed position: {results_df.iloc[i]['predicted_flux_ratios_observed_rms']:.3f}")
    print(f"Delta RMS of flux ratios at the predicted position: {results_df.iloc[i]['predicted_flux_ratios_rms']:.3f}")
    print(f"BIC: {results_df.iloc[i]['bic']:.3f}\n")

# Print LaTeX-formatted rows
print("LaTeX Table Rows:")
for row in latex_rows:
    print(row)

Model: WGD2038/SPC/outSP_point.dat
Delta RMS of Image Positions: 5.456
Delta RMS of flux ratios at the observed position: 0.148
Delta RMS of flux ratios at the predicted position: 0.144
BIC: 16.133

Model: WGD2038/SPC/outSPR_point.dat
Delta RMS of Image Positions: 0.100
Delta RMS of flux ratios at the observed position: 0.190
Delta RMS of flux ratios at the predicted position: 0.188
BIC: 20.808

Model: WGD2038/SPFC/outSPF_point.dat
Delta RMS of Image Positions: 5.621
Delta RMS of flux ratios at the observed position: 0.149
Delta RMS of flux ratios at the predicted position: 0.143
BIC: 19.760

Model: WGD2038/SPFC/outSPFR_point.dat
Delta RMS of Image Positions: 2.414
Delta RMS of flux ratios at the observed position: 0.155
Delta RMS of flux ratios at the predicted position: 0.153
BIC: 28.924

Model: WGD2038/PPC/outPP_point.dat
Delta RMS of Image Positions: 5.810
Delta RMS of flux ratios at the observed position: 0.203
Delta RMS of flux ratios at the predicted position: 0.196
BIC: 17.997
